In [1]:
import pandas as pd
import numpy as np
import os
from plotnine import *
import itertools
from PIL import Image
import math


In [2]:
path = "./data/Energy Future.csv"
df = pd.read_csv(path)

col_to_use = ['Time', 'Last', 'Source', 'Commodity']
future = df[col_to_use].copy()
future["Time"] = pd.to_datetime(future['Time'], utc=True, errors='coerce').dt.date

print(future)

             Time        Last Source  Commodity
0      2005-04-25   17.450000    EUA        EUA
1      2005-04-26   17.150000    EUA        EUA
2      2005-04-27   16.350000    EUA        EUA
3      2005-04-28   15.950000    EUA        EUA
4      2005-04-29   16.400000    EUA        EUA
...           ...         ...    ...        ...
42389  2025-11-07  114.533472    X1Z      power
42390  2025-11-07  130.963117    WWZ      power
42391  2025-11-07  132.136614    N8Z      power
42392  2025-11-07   89.120130    RVZ      power
42393  2025-11-07   59.750000    WIZ  crude oil

[42394 rows x 4 columns]


# Why Log Returns Instead of Prices?
- Raw prices across energy markets are not directly comparable:
    - Different units (MWh, MMBtu, tonnes)
    - Different price scales
    - Different volatility levels

- To address this, we compute log returns:
    - $\color{cyan}{r_t ​= \log(P_t​) − \log(P_{t−1​})}$

- Advantages of log returns
    - Scale-free and unit-independent
    - More stationary than prices
    - Additive over time
    - Standard choice in financial and energy economics

In [3]:
group_col = ["Commodity", "Source"]

future = future.sort_values(by=group_col + ['Time']).reset_index(drop=True)

print(future)

             Time     Last Source    Commodity
0      2005-04-25  17.4500    EUA          EUA
1      2005-04-26  17.1500    EUA          EUA
2      2005-04-27  16.3500    EUA          EUA
3      2005-04-28  15.9500    EUA          EUA
4      2005-04-29  16.4000    EUA          EUA
...           ...      ...    ...          ...
42389  2025-11-03   1.9161    LOZ  refined oil
42390  2025-11-04   1.9228    LOZ  refined oil
42391  2025-11-05   1.9093    LOZ  refined oil
42392  2025-11-06   1.9656    LOZ  refined oil
42393  2025-11-07   1.9403    LOZ  refined oil

[42394 rows x 4 columns]


In [4]:
future["Time"] = pd.to_datetime(future['Time'], utc=True, errors='coerce')
future["gap_days"] = future.groupby(group_col)["Time"].diff().dt.days

# 每組內計算 log return
raw_lr = future.groupby(group_col)["Last"].transform(lambda x: np.log(x) - np.log(x.shift(1)))

# 只保留 gap 在 1~7 天之間的 return，其餘設 NaN -> 
future["log_return"] = raw_lr.where(future["gap_days"].between(1, 7), np.nan)
future["Time"] = pd.to_datetime(future['Time'], utc=True, errors='coerce').dt.date
print(future)

             Time     Last Source    Commodity  gap_days  log_return
0      2005-04-25  17.4500    EUA          EUA       NaN         NaN
1      2005-04-26  17.1500    EUA          EUA       1.0   -0.017341
2      2005-04-27  16.3500    EUA          EUA       1.0   -0.047770
3      2005-04-28  15.9500    EUA          EUA       1.0   -0.024769
4      2005-04-29  16.4000    EUA          EUA       1.0    0.027823
...           ...      ...    ...          ...       ...         ...
42389  2025-11-03   1.9161    LOZ  refined oil       3.0    0.008017
42390  2025-11-04   1.9228    LOZ  refined oil       1.0    0.003491
42391  2025-11-05   1.9093    LOZ  refined oil       1.0   -0.007046
42392  2025-11-06   1.9656    LOZ  refined oil       1.0    0.029061
42393  2025-11-07   1.9403    LOZ  refined oil       1.0   -0.012955

[42394 rows x 6 columns]


# Plotting Strategy

- For each energy category:

    1. Filter data by contract code

    2.  Drop missing log returns

    3. Plot log return time series

    4. Save figures into category-specific folders

- This results in a clean structure:
    - ```
        data/plot/
            ├── gas/
            ├── power/
            ├── oil/
            ├── coal/
            └── carbon/
        ```


In [5]:
gas = {
    "TGZ":  "Dutch TTF Natural Gas",
    "XGZ":  "French PEG Natural Gas",
    "XLZ":  "Italian PSV Natural Gas",
    "JZZ":  "Austrian CEGH VTP Natural Gas",
    "NFZ":  "UK NBP Natural Gas",
    "NGZ":  "Henry Hub Natural Gas (US)",
    "CWKZ": "TTF Natural Gas ",
    "INTZ": "German THE Natural Gas"
}

power = {
    "MPZ": "Spanish Power Base",

    "N4Z": "UK Power Base",
    "N8Z": "UK Power Peak",

    "ZUZ": "Dutch Power Base",
    "ZVZ": "Dutch Power Peak",

    "X1Z": "German Power Base",
    "X2Z": "German Power Peak",

    "RUZ": "French Power Base",
    "RVZ": "French Power Peak",

    "YIZ": "Italian Power Base",
    "ZIZ": "Italian Power Peak",

    "YGZ": "Austrian Power Base",
    "ZGZ": "Austrian Power Peak",

    "WMZ": "Belgian Power Base",
    "WWZ": "Belgian Power Peak"
}

oil = {
    "CBF": "Brent Crude Oil",
    "WIZ": "WTI Crude Oil"
}

refined_oil ={
    "LFZ": "Gasoil Low Sulphur",
    "LGZ": "RBOB Gasoline Blendstock",
    "LOZ": "Heating Oil"
}

coal = {
    "LQX": "ICE Newcastle Coal",
    "LUX": "ICE Rotterdam Coal",
    "LVX": "ICE Richard Bay Coal"
}

carbon = {
    "EUA": "EU Emissions Allowance (Carbon)"
}



In [6]:
energy_sym = {
    "nature gas": gas,
    "power": power,
    "crude oil": oil,
    "refined oil": refined_oil,
    "coal": coal,
    "EUA": carbon
}

COLOR_POOL = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
    "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
    "#bcbd22", "#17becf"
]

symbol_color_map = {}

for cat, d in energy_sym.items():
    cyc = itertools.cycle(COLOR_POOL)
    for code in d.keys():
        symbol_color_map[code] = next(cyc)

In [ ]:
save_path = "./data/plot"

future['Time'] = pd.to_datetime(future['Time'])



for category, energy in energy_sym.items():
    cat_dir = os.path.join(save_path, category)
    os.makedirs(cat_dir, exist_ok=True)

    # ===== overlay (in a cate) =====
    codes = list(energy.keys())
    future_cat = future[future["Source"].isin(codes)].copy()
    # future_cat = future_cat.dropna(subset=["Log return"]) -> never do this, it will make line connect

    p = (
            ggplot(future_cat, aes(x="Time", y="log_return", color="Source"))+
            geom_line(size=0.8, alpha=0.7)+
            labs(
                title=f"{category} Log Return Over Time",
                x="Time",
                y="Log Return",
                color="Source"
            )+
            theme_bw()+
            theme(
                axis_text_x=element_text(rotation=30, hjust=1, size=8),
                figure_size=(10, 5)
            )+
            scale_x_datetime(
                date_breaks="3 months",
                date_labels="%Y-%m"
            )+
            scale_color_manual(values=symbol_color_map)
        )
    
    out_file = os.path.join(cat_dir, f"{category}_overlay_log_return.png")
    p.save(filename=out_file, dpi=300)
    print("Saved:", out_file)

    # ===== every single pic =====
    for code, name in energy.items():
        single = future[future['Source'] == code].copy()

        p = (
            ggplot(single, aes(x="Time", y="log_return"))+
            geom_line(size=0.8, color=symbol_color_map[code])+
            labs(
                title=f"{name} Log Return Over Time",
                x="Time",
                y="Log Return"
            )+
            theme_bw()+
            theme(
                axis_text_x=element_text(rotation=30, hjust=1, size=8),
                figure_size=(10, 5)
            )+
            scale_x_datetime(
                date_breaks="3 months",
                date_labels="%Y-%m"
            )
        )
        filename = f"{name}_Log_Return_Over_Time.png"
        p.save(filename=filename, path=cat_dir, dpi=300)

    print("Saved to:", os.path.join(save_path, filename))


In [ ]:
def overview_pic(cat):
    img_dir = f"./data/plot/{cat}"   
    out_path = f"./data/plot/overview/{cat}_all.png"
    os.makedirs(r"./data/plot/overview/", exist_ok=True)

    images = [
        Image.open(os.path.join(img_dir, f))
        for f in sorted(os.listdir(img_dir))
        if f.endswith(".png") & (f != f"{cat}_all.png")
    ]

    n = len(images)
    cols = math.ceil(math.sqrt(n))
    rows = math.ceil(n / cols)
    w, h = images[0].size

    canvas = Image.new("RGB", (cols * w, rows * h), "white")

    for i, img in enumerate(images):
        r = i // cols
        c = i % cols
        canvas.paste(img, (c * w, r * h))
    
    canvas.save(out_path)
    print("Saved:", out_path)

for cat, _ in energy_sym.items():
    if cat != "carbon":
        overview_pic(cat)


In [11]:
for cat, eng in energy_sym.items():
    cat_dir = os.path.join(save_path, cat)
    os.makedirs(cat_dir, exist_ok=True)
    mask = future.Commodity == cat
    merchs = future[mask]

    summary = (
        merchs.groupby("Source")
        .agg(
            data_amount=("Source", "count"),
            time_start=("Time", "min"),
            time_end=("Time", "max"),
            commodity=("Commodity", "first"),
        )
        .reset_index()
    )

    summary["time_start"] = pd.to_datetime(summary["time_start"])
    summary["time_end"]   = pd.to_datetime(summary["time_end"])

    p = (
        ggplot(summary) +
        geom_segment(
            aes(
            x="time_start",
            xend="time_end",
            y="Source",
            yend="Source"
            ),
            size=2
        ) +
        scale_x_datetime(
            date_breaks="1 year",
            date_labels="%Y"
        ) +
        labs(
            title=f"{cat} timeline",
            x="time",
            y=""
        ) +
        theme_bw() +
        theme(
            figure_size=(10, 4),
            axis_text_y=element_text(size=9)
        )
    )

    filename = f"{cat}_Timeline.png"
    # p.save(filename=filename, path=cat_dir, dpi=300)

    last_start = summary["time_start"].max()
    most_early_end = summary["time_end"].min()

    print(summary)
    # print(f'Last start in {cat} is: {last_start}')
    # print(f'Most early end in {cat} is: {most_early_end}')
    print(f"\n")

  Source  data_amount time_start   time_end   commodity
0   CWKZ           58 2025-08-20 2025-11-07  nature gas
1   ITNZ          790 2022-08-15 2025-11-07  nature gas
2    JZZ         1100 2021-08-05 2025-11-07  nature gas
3    NFZ         1681 2019-04-03 2025-11-07  nature gas
4    NGZ         3256 2012-12-03 2025-11-07  nature gas
5    TGZ         1662 2019-06-04 2025-11-07  nature gas
6    XGZ          579 2023-08-10 2025-11-07  nature gas
7    XLZ          579 2023-08-10 2025-11-07  nature gas


   Source  data_amount time_start   time_end commodity
0     MPZ          634 2023-05-24 2025-11-07     power
1     N4Z         1190 2021-04-01 2025-11-07     power
2     N8Z         1190 2021-04-01 2025-11-07     power
3     RUZ          874 2022-06-17 2025-11-07     power
4     RVZ          874 2022-06-17 2025-11-07     power
5     WMZ          635 2023-05-23 2025-11-07     power
6     WWZ          635 2023-05-23 2025-11-07     power
7     X1Z          874 2022-06-17 2025-11-07     power

In [13]:
def all_timeline(df, save_name, save_path):
    summary = (
        df.groupby("Source")
        .agg(
            data_amount=("Source", "count"),
            time_start=("Time", "min"),
            time_end=("Time", "max"),
            commodity=("Commodity", "first"),
        )
        .reset_index()
    )

    summary["time_start"] = pd.to_datetime(summary["time_start"])
    summary["time_end"]   = pd.to_datetime(summary["time_end"])
    cut_date = pd.to_datetime("2022-06-07")

    p = (
        ggplot(summary) +
        geom_segment(
            aes(
            x="time_start",
            xend="time_end",
            y="Source",
            yend="Source"
            ),
            size=2
        ) +
        geom_vline(
            xintercept=cut_date,
            linetype="dashed",
            color="red",
            size=1
        ) +
        scale_x_datetime(
            date_breaks="1 year",
            date_labels="%Y"
        ) +
        labs(
            title=f"Timeline",
            x="time",
            y=""
        ) +
        theme_bw() +
        theme(
            figure_size=(10, 4),
            axis_text_y=element_text(size=9)
        )
    )

    filename = save_name + ".png"
    # p.save(filename=filename, path=save_path+"\overview", dpi=300)

all_timeline(future,"All_Timeline", save_path)

In [ ]:
# drop last start after 2022/6/7
t1 = pd.to_datetime("2022/6/7")
t2 = pd.to_datetime("2025/11/5")

summary = (
        future.groupby("Source")
        .agg(
            data_amount=("Source", "count"),
            time_start=("Time", "min"),
            time_end=("Time", "max"),
            commodity=("Commodity", "first"),
        )
        .reset_index()
    )

keep_sources = summary.loc[(summary["time_start"] < t1), "Source"]

summary_keep = summary[summary["Source"].isin(keep_sources)].reset_index().copy()


p = (
        ggplot(summary_keep) +
        geom_segment(
            aes(
            x="time_start",
            xend="time_end",
            y="Source",
            yend="Source"
            ),
            size=2
        ) +
        geom_vline(
            xintercept=t1,
            linetype="dashed",
            color="red"
        ) +
        scale_x_datetime(
            date_breaks="1 year",
            date_labels="%Y"
        ) +
        labs(
            title=f"Timeline",
            x="time",
            y=""
        ) +
        theme_bw() +
        theme(
            figure_size=(10, 4),
            axis_text_y=element_text(size=9)
        )
    )

filename = "Reduced_All_Timeline" + ".png"
p.save(filename=filename, path=save_path+"\overview", dpi=300)


In [15]:
# Find the data should drop
out_sources = summary.loc[(summary["time_start"] > t1), "Source"]
summary_out = summary[summary["Source"].isin(out_sources)].reset_index().copy()
print(summary_out["Source"])

0     CWKZ
1     ITNZ
2      MPZ
3      RUZ
4      RVZ
5      WMZ
6      WWZ
7      X1Z
8      X2Z
9      XGZ
10     XLZ
Name: Source, dtype: object


In [ ]:
list_drop =summary_out["Source"].tolist()
re_future = future[~future["Source"].isin(list_drop)].copy()
re_future = re_future[(re_future["Time"]>=t1) & (re_future["Time"]<=t2)]

gap_list = ["YIZ", "ZIZ", "YGZ", "ZGZ"]
re_future = re_future[~future["Source"].isin(gap_list)]

print(re_future)

In [16]:
# separate data by diff merchandise
wide_by_commodity = {}
path = r"./data/merchandise"
base_dir = os.path.join(path)
os.makedirs(base_dir, exist_ok=True)

for c in re_future["Commodity"].unique():
    mer = (
        re_future[re_future["Commodity"] == c]
        .pivot(index="Time", columns="Source", values="log_return")
        .sort_index()
        .dropna(axis=0)
    )

    # 存進 dict（之後 ICA 直接用）
    wide_by_commodity[c] = mer

    # 檔名安全化（空白 → 底線）
    safe_name = c.replace(" ", "_")
    out_path = os.path.join(base_dir, f"{safe_name}.csv")
    mer.to_csv(out_path)

    print(f"Saved: {out_path}  shape={mer.shape}")

Saved: ./data/merchandise\EUA.csv  shape=(879, 1)
Saved: ./data/merchandise\coal.csv  shape=(883, 3)
Saved: ./data/merchandise\crude_oil.csv  shape=(883, 2)
Saved: ./data/merchandise\nature_gas.csv  shape=(847, 4)
Saved: ./data/merchandise\power.csv  shape=(879, 4)
Saved: ./data/merchandise\refined_oil.csv  shape=(883, 3)


In [24]:
wide_lr = (
    re_future
    .pivot(index="Time", columns="Source", values="log_return")
    .sort_index()
)

wide_lr = wide_lr.dropna(axis=0) # drop na instead of repleace with zero
print(wide_lr)


path = os.path.join( r"./data/log_return.csv")
wide_lr.to_csv(path)

print(f"Saved: {path}  shape={wide_lr.shape}")

Source           CBF       EUA       JZZ       LFZ       LGZ       LOZ  \
Time                                                                     
2022-06-07  0.018885 -0.001475 -0.010624 -0.011414  0.024391  0.024391   
2022-06-08  0.008434 -0.018620 -0.011238  0.010152  0.004721  0.004721   
2022-06-09  0.006621  0.014924  0.131576  0.000631 -0.002899 -0.002899   
2022-06-10 -0.006997  0.010438 -0.014679 -0.005694 -0.005824 -0.005824   
2022-06-13  0.009609 -0.003917 -0.009031 -0.007324 -0.000741 -0.000741   
...              ...       ...       ...       ...       ...       ...   
2025-10-30  0.000777 -0.001778 -0.026564  0.004274  0.001004  0.001004   
2025-10-31  0.006195 -0.001654 -0.010115 -0.004988  0.004323  0.004323   
2025-11-03  0.001851  0.033307  0.014499  0.010657  0.008017  0.008017   
2025-11-04 -0.006959  0.013577  0.022737 -0.002477  0.003491  0.003491   
2025-11-05 -0.014380 -0.010012 -0.011796  0.021378 -0.007046 -0.007046   

Source           LQX       LUX       